In [ ]:
import os,math
import rasterio
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import xarray as xr
import rioxarray as rxr
import geopandas as gpd
from glob import glob
from netCDF4 import Dataset
from functools import partial
from pyproj import CRS
from pyproj import Transformer
from shapely.geometry import Point, mapping
from shapely.ops import transform
from windrose import WindroseAxes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from adjustText import adjust_text

In [ ]:
def drct2val_90(x): # angle range is 90
    # within list: [upwind lower, upwind upper, downwind lower, downwind upper]
    if x == 'E':
        return [315,45,135,225]
    if x == 'NE':
        return [0,90,180,270]
    if x == 'N':
        return [45,135,225,315]
    if x == 'NW':
        return [90,180,270,360]
    if x == 'W':
        return [135,225,315,45]
    if x == 'SW':
        return [180,270,0,90]
    if x == 'S':
        return [225,315,45,135]
    if x == 'SE':
        return [270,360,90,180]
    
def ang_mat(basemat):
    ## create angle matrix 
    ## basemat need to be in the form opened by rioxarray.open_rasterio 
    
    ## be careful with the dimension name in any opened tif, they need to be consistent!
    center_x = int(np.floor(basemat.x.size/2))
    center_y = int(np.floor(basemat.y.size/2))
    xmat = np.array([[x-basemat.x[center_x].values for x in basemat.x.values]]*len(basemat.y.values))
    ymat = np.array([[y-basemat.y[center_y].values for y in basemat.y.values]]*len(basemat.x.values)).T
    arr_ang = np.rad2deg(np.arctan2(ymat,xmat))
    arr_ang = np.array(arr_ang<0)*360+arr_ang
        
    ang_buffer = xr.DataArray(
             data = arr_ang,
             dims = ['y','x'], # control the x,y layout when plotting
             coords = dict(
               x = (['x'],basemat.x.values),
               y = (['y'],basemat.y.values),
               )
            )
    return ang_buffer

In [ ]:
start_yr = 2015
end_yr = 2021
ls_city = ['Atlanta','Austin','Baltimore','Charlotte','Cincinnati','Columbus','Dallas','DC','Houston','Indianapolis','KC','Louisville','Memphis','Miami','Minneapolis','Nashville','NYC','OKC','Omaha','Orlando',
           'Phoenix','Pittsburgh','Richmond','SanAntonio','StLouis','Tucson','Philadelphia']
# 
ls_lon = [-84.3880,-97.7431,-76.6122,-80.8431,-84.5120,-82.9988,-96.7970,-77.0369,-95.3698,-86.1581,-94.5786,-85.7585,-90.0490,-80.1918,-93.2650,-86.7816,-74.0060,-97.5164,-95.9345,
          -81.3789,-112.0740,-79.9959,-77.4360,-98.4936,-90.1994,-110.9747,-75.1652]
#
ls_lat = [33.7490,30.2672,39.2904,35.2271,39.1031,39.9612,32.7767,38.9072,29.7604,39.7684,39.0997,38.2527,35.1495,25.7617,44.9778,36.1627,40.7128,35.4676,41.2565,28.5384,
          33.4484,40.4406,37.5407,29.4241,38.6270,32.2226,39.9526]

all_drcts = ['E','NE','N','NW','W','SW','S','SE'] 

df_city = pd.DataFrame({'Name':ls_city,'lon':ls_lon,'lat':ls_lat})
df_city = df_city.sort_values('Name',ignore_index=True)

# DEI at different directions

In [ ]:
## for each city, rank wind directions and get DEI (with mapping)
os.chdir('/glade/scratch/USERNAME/MRMS_new/')

if not os.path.exists('Results/Imgs'): ## store plottings
    os.mkdir('Results/Imgs')

## open wind direction ranking data
wnd_rank = pd.read_csv('Results/8PrevailDrcts.csv',sep=' ')
## open DEI results per wind direction
dat_DEI = pd.read_csv('Results/DEI_AllDrct90.csv',sep=' ')

plt.style.use(['science','no-latex'])

for city in df_city['Name']:
    drct_ord = wnd_rank[wnd_rank['City']==city].iloc[:,2:] # iloc[:,2:] means select all col values from 3rd col to the end
    drct_ord = drct_ord.values[0]  # in list form
    
    ls_DEI = [] # store DEI values based on wind direction sequence, for each city
    ls_Map = [] # store precip map for each direction
    imgs = glob('Results/Directional Hourly Map/'+city+'/*.tif')
    
    for i in drct_ord:
        DEI = dat_DEI[dat_DEI['City']==city][i].values[0]
        ls_DEI.append(DEI)
        
        # open corresponding tif image
        for j in imgs:
            if j.split('_')[1].split('.')[0] == i:
                img = rxr.open_rasterio(j)[0]
                img = (img-img.mean())/img.std() # standardization
                ls_Map.append(img)
    
    # for unifying colorbars
    ls_max = [i.max().values for i in ls_Map]
    max_val = np.ceil(max(ls_max))
    ls_min = [i.min().values for i in ls_Map]
    min_val = np.floor(min(ls_min))
    
    fig,ax = plt.subplots(2,4,figsize=(12,6))
    
    ang_buf = ang_mat(ls_Map[0])
    
    count = 0
    for idx,item in enumerate(ls_Map):
        
        if drct_ord[idx] == 'E':
            item = item.where(((ang_buf>=drct2val_90(drct_ord[idx])[0])|(ang_buf<=drct2val_90(drct_ord[idx])[1]))| \
                              ((ang_buf>=drct2val_90(drct_ord[idx])[2])&(ang_buf<=drct2val_90(drct_ord[idx])[3])))
        elif drct_ord[idx] == 'W':
            item = item.where(((ang_buf>=drct2val_90(drct_ord[idx])[0])&(ang_buf<=drct2val_90(drct_ord[idx])[1]))| \
                              ((ang_buf>=drct2val_90(drct_ord[idx])[2])|(ang_buf<=drct2val_90(drct_ord[idx])[3])))
        else:
            item = item.where(((ang_buf>=drct2val_90(drct_ord[idx])[0])&(ang_buf<=drct2val_90(drct_ord[idx])[1]))| \
                              ((ang_buf>=drct2val_90(drct_ord[idx])[2])&(ang_buf<=drct2val_90(drct_ord[idx])[3])))
        
        ax_img = ax[int(idx/4),idx%4].imshow(item,cmap='RdBu_r',origin='lower',vmin=min_val,vmax=max_val)
        #divider = make_axes_locatable(ax[int(idx/4),idx%4])
        #cax = divider.append_axes("right", size="5%", pad=0.15)
        #fig.colorbar(ax_img, cax=cax)
        ax[int(idx/4),idx%4].set_xticklabels([])
        ax[int(idx/4),idx%4].set_yticklabels([])
        ax[int(idx/4),idx%4].set_xlabel('DEI: '+str(ls_DEI[idx]),size=12)
        ax[int(idx/4),idx%4].set_title(drct_ord[idx],size=12)
        count = count+1
    fig.colorbar(ax_img,ax=ax.ravel().tolist(),shrink=0.8)
    fig.text(0.08,0.3,'Normalized Precipitation',rotation=90,fontsize=14)
    #if count == len(ls_Map):
    #    plt.savefig('Results/Imgs/'+city+'_plot.png')

# ring-based standardized precipitation

In [ ]:
os.chdir('/glade/scratch/USERNAME/MRMS_new/Results/RingPrecip')
if not os.path.exists('../Imgs/Precip_Ring'): ## store plottings
    os.mkdir('../Imgs/Precip_Ring')

plt.style.use(['science','no-latex'])

ls_file = sorted(glob('*.csv'))
ls_dis = np.arange(-55,65,10)

for item in ls_file:
    
    city = item.split('_')[0]
    
    fig,ax = plt.subplots(2,4,figsize=(16,6))
    
    dat = pd.read_csv(item, sep=' ')
    
    for i in range(dat.shape[0]):
        res = dat.iloc[i,:].tolist()
        ax[int(i/4),i%4].set_title(res[1],size=14)
        ax[int(i/4),i%4].plot(ls_dis,res[2:14])
        ax[int(i/4),i%4].scatter(ls_dis,res[2:14])
        
    fig.text(0.08,0.23,'Normalized Ring-based Precipitation',rotation=90,fontsize=14)
    fig.text(0.45,0.05,'Distance to Center',fontsize=14)
    
    plt.savefig('../Imgs/Precip_Ring/'+city+'_ring.png')

# DEI vs. City Size (dom/non-dom)

In [ ]:
os.chdir('/glade/scratch/USERNAME/MRMS_new')

# read dataframe containing city sizes
dat_urban = pd.read_csv('../Modis/df_urban2.csv',sep = ' ')
dat_urban = dat_urban.sort_values('City',ignore_index = True)

# read DEI at dominant and non-dominant directions
dat_DEI = pd.read_csv('Results/DEI_Dom_NonDom.csv',sep=' ')

dat_urban = dat_urban.merge(dat_DEI,how='outer',on='City')

plt.style.use(['science','no-latex'])
fig,ax = plt.subplots(1,2,figsize=(15,6))

ax[0].scatter(dat_urban[dat_urban['Significance']=='Non-Sig']['FullSize05']/2,
             dat_urban[dat_urban['Significance']=='Non-Sig']['PrevDrct(2)'],alpha=0.7)
ax[0].scatter(dat_urban[dat_urban['Significance']=='Sig']['FullSize05']/2,
             dat_urban[dat_urban['Significance']=='Sig']['PrevDrct(2)'],c='r',alpha=0.7)

ax[1].scatter(dat_urban[dat_urban['Significance']=='Non-Sig']['FullSize05']/2,
             dat_urban[dat_urban['Significance']=='Non-Sig']['NonPrevDrct(6)'],alpha=0.7)
ax[1].scatter(dat_urban[dat_urban['Significance']=='Sig']['FullSize05']/2,
             dat_urban[dat_urban['Significance']=='Sig']['NonPrevDrct(6)'],c='r',alpha=0.7)

#texts = [ax[0].text(dat_urban['FullSize05'].values[i]/2,dat_urban['PrevDrct(2)'].values[i],
#                 dat_urban['City'].values[i],size=6) for i in range(len(dat_urban['City'].values))]
#adjust_text(texts)

#texts = [ax[1].text(dat_urban['FullSize05'].values[i]/2,dat_urban['NonPrevDrct(6)'].values[i],
#                 dat_urban['City'].values[i],size=6) for i in range(len(dat_urban['City'].values))]
#adjust_text(texts)

ax[0].axhline(y=0,color='k',ls='--')
ax[1].axhline(y=0,color='k',ls='--')

ax[0].set_xlabel('Urban Size (km)',size = 16)
ax[1].set_xlabel('Urban Size (km)',size = 16)
ax[0].set_ylabel('DEI at Prevailing Directions',size = 16)
ax[1].set_ylabel('DEI at Non-Prevailing Directions',size = 16)

# Heatmap (x: dis to center, y: imp level, val: normed precip)

In [ ]:
os.chdir('/glade/scratch/USERNAME/MRMS_new/Results')

plt.style.use(['science','no-latex'])

for city in df_city['Name']:
    imp = pd.read_csv('Modis/Ring_Result/'+city+'_RingImp.csv',sep=' ')
    precip = pd.read_csv('RingPrecip/'+city+'_RingPrecip.csv',sep=' ')
    
    df = pd.DataFrame()
    
    fig,ax = plt.subplots(2,4,figsize=(20,6),sharex=True)
    
    dis = precip.columns.values[2:]
    dis = [x.split('_')[0] for x in dis]
    dis = [int(x)+5 for x in dis]
    
    ls_df = []  # list to store pivot table for each direction
    drcts = []
    for i in range(8):
        drcts.append(precip.iloc[i].values[1])
        ls_precip = precip.iloc[i].values[2:]
        ls_imp = imp.iloc[i].values[2:]
        ls_imp = [round(x,2) for x in ls_imp]
        imp_class = pd.cut(ls_imp,bins=np.arange(0,11,1)/10,
                           labels = ['0-0.1','0.1-0.2','0.2-0.3','0.3-0.4','0.4-0.5','0.5-0.6','0.6-0.7','0.7-0.8','0.8-0.9','0.9-1'])
        tmpdf = pd.DataFrame({'Distance':dis,'Imp':ls_imp,'ImpClass':imp_class,'Precip':ls_precip})
        tmpdf = tmpdf.pivot_table(index='Imp',columns='Distance',values='Precip')
        ls_df.append(tmpdf)
    
    for idx in range(8):
        sns.heatmap(ls_df[idx],ax=ax[int(idx/4),idx%4],cmap='RdBu_r',center=0,
                    cbar_kws={'format': '%.1f'})
        ax[int(idx/4),idx%4].invert_yaxis()
        ax[int(idx/4),idx%4].set_title(drcts[idx])
        if idx<4:
            ax[int(idx/4),idx%4].set(xlabel=None)
        else:
            ax[int(idx/4),idx%4].set_xlabel('Distance to Center (km)',size=12)
        if idx%4 == 0:
            ax[int(idx/4),idx%4].set_ylabel('Imperviousness',size=12)
        else:
            ax[int(idx/4),idx%4].set(ylabel=None)
    
    fig.suptitle('Standardized Precipitation VS. Imperviousness: '+city,size=18)
    
    fig.savefig('Imgs/Heatmap/'+city+'_DisImp.png',dpi=300)